In [1]:
import pandas as pd
import numpy as np
from importlib import reload
import HateSpeechAggregator as hsa
reload(hsa)
from glob import glob
from scipy.stats import chi2_contingency


files = glob('/home/bupi/Documents/pdy/hs/dashboard/clean/comments/*.csv')
files.sort()
print(len(files))

# res = hsa.load_df(files[0], pd.datetime(2020,1,1,), pd.datetime(2020,12,12))

8


In [20]:
df = hsa.load_df(files[0], pd.datetime(2020,1,1), pd.datetime(2020,12,12))

In [21]:
df.head(2)

,profile_id,datetime,likes,comment_message,lex_found,post_url,topics_found,page_name,lex_found_post
0,100004238449395,2020-03-24 13:01:38,1,အမေ စု သက် ရှည် ကျန်းမာ ပါစေ 🙏,NaN,https://www.facebook.com/7daynews/posts/315261...,[],7Day News Journal,NaN
1,100046335742147,2020-03-24 13:02:29,2,Educatingpeopleisessential..Teachpeopletodofre...,NaN,https://www.facebook.com/7daynews/posts/315261...,[],7Day News Journal,NaN


In [4]:
f = files[0]
cmt = pd.read_csv(f)
pst = pd.read_csv(f.replace('comments','posts'))
print(cmt.shape, pst.shape)
jnd = cmt.merge(pst, on='post_url', how='left', suffixes=('_cmt','_pst'))

(58725, 6) (400, 13)


In [5]:
print(jnd.columns)

Index(['profile_id', 'datetime', 'likes_cmt', 'comment_message',
       'lex_found_cmt', 'post_url', 'page_name', 'likes_pst', 'comments',
       'shares', 'love', 'wow', 'haha', 'sad', 'angry', 'thankful',
       'post_message', 'lex_found_pst'],
      dtype='object')


In [23]:
jnd['has_lex'] = ~jnd.lex_found_x.isna()
lex_count = jnd.groupby('page_name')['has_lex'].sum()
total = jnd.groupby('page_name')['page_name'].count()
total.name = 'total'
page_hs_ratio = pd.DataFrame([lex_count, total]).T
page_hs_ratio['ratio'] = page_hs_ratio.has_lex / page_hs_ratio.total
page_hs_ratio.sort_values('ratio', ascending=False, inplace=True)
page_hs_ratio.head()

,has_lex,total,ratio
page_name,,,
Development Media Group,101.0,421.0,0.239905
Khit Thit Media,362.0,2658.0,0.136193
သတင်းပေါင်းချုပ်,1.0,8.0,0.125000
The Ladies News,54.0,539.0,0.100186
Kumudra,135.0,1719.0,0.078534


In [8]:
posts_wlex = jnd.loc[~jnd.lex_found_pst.isna(), :]
posts_nlex = jnd.loc[jnd.lex_found_pst.isna(), :]
print(posts_nlex.shape, posts_wlex.shape)

(54766, 18) (3959, 18)


In [12]:
pwl_cwl = posts_wlex.lex_found_cmt.count()
pwl_cnl = posts_wlex.shape[0] - pwl_cwl
pnl_cwl = posts_nlex.lex_found_cmt.count()
pnl_cnl = posts_nlex.shape[0] - pnl_cwl
hspost_effect_df = pd.DataFrame({
    'has_lex': [True, False],
    'number_of_pages': [posts_wlex.shape[0], posts_nlex.shape[0]],
    'comment_wlex': [pwl_cwl, pnl_cwl],
    'cmement_nlex': [pwl_cnl, pnl_cnl]
})
hspost_effect_df

,has_lex,number_of_pages,comment_wlex,cmement_nlex
0,True,3959,880,3079
1,False,54766,1203,53563


In [32]:
x = pd.read_csv('/home/bupi/Documents/pdy/hs/dashboard/clean/aggregated/20200420115700/hspost-effect.csv')
x = x.groupby(['post_has_lex','comment_has_lex'])['count'].sum()
y = chi2_contingency(x.unstack())

In [34]:
y[1]

0.0

In [5]:
x = pd.read_csv(files[0])
x.head(2)

,post_id,c_id,n_id,profile_id,datetime,likes,comment_message,lex_found,post_url
0,25032020_0072,1,0,100004238449395,2020-03-24 13:01:38,1,အမေ စု သက် ရှည် ကျန်းမာ ပါစေ 🙏,NaN,https://www.facebook.com/7daynews/posts/315261...
1,25032020_0072,3,0,100046335742147,2020-03-24 13:02:29,2,Educatingpeopleisessential..Teachpeopletodofre...,NaN,https://www.facebook.com/7daynews/posts/315261...


In [8]:
reply_df = x.groupby(['post_id','c_id'])['c_id'].count()
print(reply_df.shape)
reply_df = reply_df[reply_df>1]
print(reply_df.shape)

(49243,)
(3307,)


In [33]:
pc_ids = []
for (post_id,c_id),_ in reply_df.iteritems():
    pc_ids.append(post_id + str(c_id).zfill(4))
pc_ids = set(pc_ids)
print(len(pc_ids))
x['pc_id'] = [p+str(c).zfill(4) for p,c in zip(x.post_id, x.c_id)]
x.head(2)

3307


,post_id,c_id,n_id,profile_id,datetime,likes,comment_message,lex_found,post_url,pc_id
0,25032020_0072,1,0,100004238449395,2020-03-24 13:01:38,1,အမေ စု သက် ရှည် ကျန်းမာ ပါစေ 🙏,NaN,https://www.facebook.com/7daynews/posts/315261...,25032020_00720001
1,25032020_0072,3,0,100046335742147,2020-03-24 13:02:29,2,Educatingpeopleisessential..Teachpeopletodofre...,NaN,https://www.facebook.com/7daynews/posts/315261...,25032020_00720003


In [35]:
repdf = x.loc[x.pc_id.isin(pc_ids)]
print(repdf.shape)
display(repdf.head(2))

(12789, 10)


,post_id,c_id,n_id,profile_id,datetime,likes,comment_message,lex_found,post_url,pc_id
3,25032020_0072,5,0,100044448832921,2020-03-24 13:03:50,12,အပြော က စကော လက်တွေ့ က ဇီး ရွက်,NaN,https://www.facebook.com/7daynews/posts/315261...,25032020_00720005
4,25032020_0072,5,1,100045388926874,2020-03-24 13:14:18,6,AungSoe မင်း ပထွေး စစ်ခွေး နဲ့ တော့ ကွာ တယ်,စစ်ခွေး,https://www.facebook.com/7daynews/posts/315261...,25032020_00720005


## split first comments into 2--wl and nl

In [36]:
nis0 = repdf.loc[repdf.n_id==0]
print(nis0.shape)

(3263, 10)


In [37]:
first_comment_nl = nis0.loc[nis0.lex_found.isna()]
first_comment_wl = nis0.loc[~nis0.lex_found.isna()]
print(first_comment_wl.shape, first_comment_nl.shape)

(198, 10) (3065, 10)


In [41]:
first_cwl = repdf.loc[np.logical_and(repdf.pc_id.isin(first_comment_wl.pc_id), repdf.n_id>0),:]
first_cnl = repdf.loc[np.logical_and(~repdf.pc_id.isin(first_comment_wl.pc_id), repdf.n_id>0),:]
print(first_cwl.shape, first_cnl.shape)
# first_comment_wl.head()

(680, 10) (8846, 10)


In [42]:
fcwl_nwl = first_cwl.lex_found.count()
fcwl_nnl = first_cwl.shape[0] - fcwl_nwl
fcnl_nwl = first_cnl.lex_found.count()
fcnl_nnl = first_cnl.shape[0] - fcnl_nwl
hsfirst_comment_effect = pd.DataFrame({
    'first_comment_has_lex': ['First Comment with HSL','First Comment with HSL','First Comment with no HSL','First Comment with no HSL'],
    'reply_has_lex': ['Reply with HSL','Reply with no HSL','Reply with HSL','Reply with no HSL'],
    'count': [fcwl_nwl, fcwl_nnl, fcnl_nwl, fcnl_nnl]
})
hsfirst_comment_effect

,first_comment_has_lex,reply_has_lex,count
0,First Comment with HSL,Reply with HSL,132
1,First Comment with HSL,Reply with no HSL,548
2,First Comment with no HSL,Reply with HSL,352
3,First Comment with no HSL,Reply with no HSL,8494


In [50]:
chi2_contingency(hsfirst_comment_effect.groupby(['first_comment_has_lex','reply_has_lex'])['count'].sum().unstack())

(308.6489116428305,
 4.3008296107630825e-69,
 1,
 array([[  34.54965358,  645.45034642],
        [ 449.45034642, 8396.54965358]]))

reply_has_lex,Reply with HSL,Reply with no HSL
first_comment_has_lex,,
First Comment with HSL,132,548
First Comment with no HSL,352,8494


In [52]:
y = pd.read_csv('/home/bupi/Documents/pdy/hs/dashboard/clean/posts/20200323_20200325.csv')

In [53]:
x.merge(y)

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [56]:
pd.concat((x,y), join='left', on='post_url')

TypeError: concat() got an unexpected keyword argument 'on'

In [54]:
x.head(2)

,post_id,c_id,n_id,profile_id,datetime,likes,comment_message,lex_found,post_url,pc_id
0,25032020_0072,1,0,100004238449395,2020-03-24 13:01:38,1,အမေ စု သက် ရှည် ကျန်းမာ ပါစေ 🙏,NaN,https://www.facebook.com/7daynews/posts/315261...,25032020_00720001
1,25032020_0072,3,0,100046335742147,2020-03-24 13:02:29,2,Educatingpeopleisessential..Teachpeopletodofre...,NaN,https://www.facebook.com/7daynews/posts/315261...,25032020_00720003


In [55]:
y.head(2)

,page_name,likes,comments,shares,love,wow,haha,sad,angry,thankful,post_url,post_message,lex_found
0,AKonThi,https://www.facebook.com/AKonThiNEWS/posts/117...,33769,858,57029,1256,44,80,35,7,0,နိုင်ငံတော် ၏ အတိုင်ပင်ခံ ပုဂ္ဂိုလ် ဒေါ် အောင်...,NaN
1,MOI Webportal Myanmar,https://www.facebook.com/MOIWebportalMyanmar/p...,4605,272,9095,13,382,3,1949,5,0,COVID-19 ရောဂါ စောင့်ကြပ် ကြည့်ရှု မှု နှင့် ပ...,NaN


## Page Outreach

In [1]:
df = pd.read_csv(files[0], usecols=['profile_id','date','post_url'])
df['page'] = df.post_url.apply(lambda x: x.split('posts')[0])
df.head(2)

NameError: name 'pd' is not defined

In [6]:
df.groupby('page')['profile_id'].nunique()

page
https://www.facebook.com/208118176698616/         69
https://www.facebook.com/597015637154804/         51
https://www.facebook.com/714912165290510/          6
https://www.facebook.com/7daynews/              1543
https://www.facebook.com/AKonThiNEWS/           1343
                                                ... 
https://www.facebook.com/thutakyaw.mm/            32
https://www.facebook.com/we.com.mm/              221
https://www.facebook.com/www.thadinsar.site/       8
https://www.facebook.com/yuyaonlinethadin/         9
https://www.facebook.com/zomipost.news/          632
Name: profile_id, Length: 76, dtype: int64

In [7]:
df.profile_id[df.page=='https://www.facebook.com/7daynews/'].unique().shape

(1543,)

## hs ratio page

In [3]:
comment_df = pd.read_csv(files[0]).merge(
    pd.read_csv(files[0].replace('comments','posts')),
    on='post_url',
    suffixes=('','_post')
)
comment_df.head(2)

,post_id,c_id,n_id,profile_id,datetime,likes,comment_message,lex_found,post_url,page_name,...,shares,love,wow,haha,sad,angry,thankful,likes_post,post_message,lex_found_post
0,25032020_0072,1,0,100004238449395,2020-03-24 13:01:38,1,အမေ စု သက် ရှည် ကျန်းမာ ပါစေ 🙏,NaN,https://www.facebook.com/7daynews/posts/315261...,7Day News Journal,...,473,7251,357,7,28,9,1,0,ပြည်တွင်း မှာ COVID-19 ရောဂါပိုး ရှိ သူ နှစ် ဦ...,NaN
1,25032020_0072,3,0,100046335742147,2020-03-24 13:02:29,2,Educatingpeopleisessential..Teachpeopletodofre...,NaN,https://www.facebook.com/7daynews/posts/315261...,7Day News Journal,...,473,7251,357,7,28,9,1,0,ပြည်တွင်း မှာ COVID-19 ရောဂါပိုး ရှိ သူ နှစ် ဦ...,NaN


In [7]:
comment_df['datetime'] = pd.to_datetime(comment_df.datetime)
comment_df['date'] = [pd.datetime(d.year, d.month, d.day) for d in comment_df.datetime]
comment_df['has_lex'] = ~comment_df.lex_found.isna()
lex_count = comment_df.groupby(['page_name','date'])['has_lex'].sum()
total = comment_df.groupby(['page_name','date'])['page_name'].count()
total.name = 'total'
page_hs_ratio_df = pd.DataFrame([lex_count, total]).T
page_hs_ratio_df['ratio'] = page_hs_ratio_df.has_lex / page_hs_ratio_df.total

In [8]:
page_hs_ratio_df

has_lex   total     ratio
page_name           date                                 
7Day News Journal   2020-03-23      2.0    91.0  0.021978
                    2020-03-24     47.0  1524.0  0.030840
                    2020-03-25      3.0   148.0  0.020270
AKonThi             2020-03-23     17.0   434.0  0.039171
                    2020-03-24     12.0  1000.0  0.012000
...                                 ...     ...       ...
သတင်းထူးသတင်းဦးများ 2020-03-23      1.0   126.0  0.007937
                    2020-03-24      0.0    56.0  0.000000
                    2020-03-25      0.0     5.0  0.000000
သတင်းပေါင်းချုပ်    2020-03-23      1.0     2.0  0.500000
                    2020-03-24      0.0     6.0  0.000000

[195 rows x 3 columns]

In [6]:
df = pd.read_csv('/home/bupi/Documents/pdy/hs/dashboard/clean/aggregated/20200421221839/page-hs-ratio.csv', sep='~')
df['date'] = pd.to_datetime(df.date)

In [13]:
df = df.loc[df.date==pd.datetime(2020,4,10),:]
df.total.sum(), df.has_lex.sum()

(175.0, 6.0)

In [15]:
print(files[-2])
df = pd.read_csv(files[-2])
df.shape

/home/bupi/Documents/pdy/hs/dashboard/clean/comments/20200409_20200413.csv


(125176, 9)

In [16]:
df.

,post_id,c_id,n_id,profile_id,datetime,likes,comment_message,lex_found,post_url
0,20200409_0119,1,0,100015150585131,2020-04-15 07:27:34,3,ပိတ် လိုက် တော့ လေ ကိုယ်ပူ တိုင်းတာ ဘယ်လောက် ထ...,NaN,https://www.facebook.com/2206963259620238/post...
1,20200409_0119,2,0,100003563438479,2020-04-15 07:28:20,2,Interviewer အစ်ကို ရေ 😷 Mask ကို စနစ်တကျ သုံးစ...,NaN,https://www.facebook.com/2206963259620238/post...
